# Multi-head classification with AutoKeras

## Input
Functional connectivity of the size N_networks * N_networks. The connectivity matrix is symmetric so it will be flattened and only upper traingle (k=1) will be kept.

## Output
There are two outputs:
1. Participant rating of their hypnotic depth (1 = hypnotized OR 0 = non-hypnotized)
2. participants hypnotizability score (low OR high)

In [1]:
import pandas as pd
import tensorflow as tf
import autokeras as ak

In [2]:
DATA = pd.read_csv('data/connectivities_yeo17nets.csv', index_col=0)

# add the hypnotizability data
hypnotizability = pd.read_excel('data/PLB_HYP_data_MASTER.xlsx', index_col=0)
hypnotizability = hypnotizability['hypnotizability_total'].iloc[9:59]
DATA['hypnotizability_total'] = hypnotizability.repeat(4).reset_index(drop=True)
# drop rows with nan values
DATA = DATA.dropna(axis=0, how='any').reset_index(drop=True)

# keep the blocks where procedure is described as hypnosis
DATA = DATA.query('description == "hypnosis"')
DATA = DATA.query('procedure == "confusion" or procedure == "relaxation"')

X = DATA.drop(columns=['hypnosis_depth', 'procedure', 'description', 'session', 'condition',
                       'bids_id', 'hypnotizability_total'])


y_depth = DATA['hypnosis_depth'].apply(lambda x: 1 if x > 5 else 0)
y_score = DATA['hypnotizability_total'].apply(lambda x: 'low' if x <= 5 else 'high')
y_depth.value_counts(normalize=True), y_score.value_counts(normalize=True)

(1    0.585366
 0    0.414634
 Name: hypnosis_depth, dtype: float64,
 high    0.609756
 low     0.390244
 Name: hypnotizability_total, dtype: float64)

In [ ]:
# Initialize the multi with multiple inputs and outputs.
model = ak.AutoModel(
    inputs=[ak.StructuredDataInput()],
    outputs=[
        ak.ClassificationHead(loss="categorical_crossentropy", metrics=["accuracy"]),
        ak.ClassificationHead(loss="categorical_crossentropy", metrics=["accuracy"]),
    ],
    overwrite=True,
    max_trials=10,
    project_name='temp/2_head_model'
)
# Fit the model with prepared data.
model.fit(
    [X],
    [y_depth, y_score],
)

model.evaluate([X],
    [y_depth, y_score])

In [6]:
class SingleDenseLayerBlock(ak.Block):
    def build(self, hp, inputs=None):
        # Get the input_node from inputs.
        input_node = tf.nest.flatten(inputs)[0]
        layer = tf.keras.layers.Dense(
            hp.Int("num_units", min_value=2, max_value=32, step=2)
        )
        output_node = layer(input_node)
        return output_node

In [7]:
input_node = ak.StructuredDataInput()
output_node = ak.Normalization()(input_node)
output_node = SingleDenseLayerBlock()(output_node)
output_node = SingleDenseLayerBlock()(output_node)
output_node = SingleDenseLayerBlock()(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=3
)
history = clf.fit(X, y_depth)

Trial 3 Complete [00h 00m 02s]
val_loss: 0.553253173828125

Best val_loss So Far: 0.04204084724187851
Total elapsed time: 00h 01m 48s
INFO:tensorflow:Oracle triggered exit


2022-05-30 16:45:43.260866: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-30 16:45:43.514313: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 289ms/step - loss: 1.0395 - accuracy: 0.4878
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [4]:
clf = ak.StructuredDataClassifier(max_trials=10, overwrite=True, project_name='temp/structured_data_classifier')
history = clf.fit(X, y_depth)
clf.evaluate(X, y_depth)

Trial 1 Complete [00h 03m 34s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 03m 34s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
True              |True              |structured_data_block_1/normalize
False             |False             |structured_data_block_1/dense_block_1/use_batchnorm
1                 |2                 |structured_data_block_1/dense_block_1/num_layers
32                |32                |structured_data_block_1/dense_block_1/units_0
0                 |0                 |structured_data_block_1/dense_block_1/dropout
32                |32                |structured_data_block_1/dense_block_1/units_1
0                 |0                 |classification_head_1/dropout
adam              |adam              |optimizer
0.001             |0.001             |learning_rate



2022-05-30 16:31:06.601824: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-30 16:31:06.659730: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-30 16:31:06.714606: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-30 16:31:06.761682: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-30 16:31:06.817584: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-30 16:31:06.884535: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-30 16:31:06.935680: I tensorflow/core/grappler/optimizers/cust

Epoch 1/1000


2022-05-30 16:31:09.423954: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-30 16:31:10.843463: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.7486 - accuracy: 0.5938

2022-05-30 16:31:12.961244: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 5s 5s/step - loss: 0.7486 - accuracy: 0.5938 - val_loss: 1.4165 - val_accuracy: 0.3333
Epoch 2/1000
1/1 [==============================] - 1s 508ms/step - loss: 0.6041 - accuracy: 0.6562 - val_loss: 1.4152 - val_accuracy: 0.6667
Epoch 3/1000
1/1 [==============================] - 0s 415ms/step - loss: 0.5113 - accuracy: 0.7500 - val_loss: 1.4046 - val_accuracy: 0.5556
Epoch 4/1000
1/1 [==============================] - 0s 393ms/step - loss: 0.4432 - accuracy: 0.8750 - val_loss: 1.3829 - val_accuracy: 0.5556
Epoch 5/1000
1/1 [==============================] - 0s 345ms/step - loss: 0.3883 - accuracy: 0.9375 - val_loss: 1.3469 - val_accuracy: 0.7778
Epoch 6/1000
1/1 [==============================] - 1s 528ms/step - loss: 0.3403 - accuracy: 0.9375 - val_loss: 1.2977 - val_accuracy: 0.7778
Epoch 7/1000
1/1 [==============================] - 0s 280ms/step - loss: 0.2991 - accuracy: 0.9375 - val_loss: 1.2419 - val_accuracy: 0.7778
Epoch 8/1000
1/1 [====

KeyboardInterrupt: 

In [ ]:
model = clf.export_model()
model.summary()
print(x_train.dtype)
# numpy array in object (mixed type) is not supported.
# convert it to unicode.
model.predict(x_train.astype(np.unicode))

predicted_y = clf.predict(test_file_path)